In [75]:
%run code/import_libs.py
%run code/functions_v1.2.py
%run code/Build_Forms_v1.2.py
%run code/initial_data_prep.py

%matplotlib inline

initial_pred = df.drop(columns=["price"]).copy()
initial_price = df[["price"]]

mean_price_2014_2015=initial_price.mean()[0]

# Create OLS linear model
pred_fin, price_fin = transform_data(initial_pred, initial_price)

pred_int = sm.add_constant(pred_fin)
model = sm.OLS(price_fin,pred_int).fit()

coef_df=model.params.reset_index()
coef_df.columns=["Column","Value"]

In [76]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.881
Model:                            OLS   Adj. R-squared:                  0.880
Method:                 Least Squares   F-statistic:                     801.0
Date:                Fri, 01 Apr 2022   Prob (F-statistic):               0.00
Time:                        00:18:11   Log-Likelihood:                 6029.9
No. Observations:               21143   AIC:                        -1.167e+04
Df Residuals:                   20949   BIC:                        -1.013e+04
Df Model:                         193                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             7.8039      0.041    192.509      0.000       7.724       7.883
sqft_living       0.4536      0.005     85.940      0.000       0.443       0.464
sqft_lot          0.0773      0.002     35.661      0.000       0.073       0.082
zip_98002         0.0070      0.016      0.430      0.667      -0.025       0.039
zip_98003         0.0403      0.015      2.718      0.007       0.011       0.069
zip_98004         1.1644      0.015     80.052      0.000       1.136       1.193
zip_98005         0.7896      0.017     45.280      0.000       0.755       0.824
zip_98006         0.6976      0.013     53.382      0.000       0.672       0.723
zip_98007         0.7033      0.019     37.910      0.000       0.667       0.740
zip_98008         0.7043      0.015     47.076      0.000       0.675       0.734
zip_98010         0.2390      0.021     11.345      0.000       0.198       0.280
zip_98011         0.4748      0.016     28.824      0.000       0.442       0.507
zip_98014         0.2835      0.019     14.580      0.000       0.245       0.322
zip_98019         0.3117      0.017     18.737      0.000       0.279       0.344
zip_98022         0.0484      0.016      3.076      0.002       0.018       0.079
zip_98023         0.0034      0.013      0.266      0.791      -0.022       0.029
zip_98024         0.4013      0.023     17.443      0.000       0.356       0.446
zip_98027         0.5353      0.013     39.734      0.000       0.509       0.562
zip_98028         0.4302      0.015     29.121      0.000       0.401       0.459
zip_98029         0.6294      0.015     43.385      0.000       0.601       0.658
zip_98030         0.0587      0.015      3.867      0.000       0.029       0.089
zip_98031         0.1085      0.015      7.253      0.000       0.079       0.138
zip_98032         0.0044      0.019      0.229      0.819      -0.033       0.042
zip_98033         0.8164      0.013     61.395      0.000       0.790       0.842
zip_98034         0.5777      0.013     45.514      0.000       0.553       0.603
zip_98038         0.1776      0.013     14.198      0.000       0.153       0.202
zip_98039         1.3581      0.028     48.184      0.000       1.303       1.413
zip_98040         0.9479      0.015     62.877      0.000       0.918       0.977
zip_98042         0.0910      0.013      7.235      0.000       0.066       0.116
zip_98045         0.2987      0.016     18.711      0.000       0.267       0.330
zip_98052         0.6681      0.013     53.111      0.000       0.643       0.693
zip_98053         0.5861      0.014     43.182      0.000       0.559       0.613
zip_98055         0.1576      0.015     10.484      0.000       0.128       0.187
zip_98056         0.3534      0.013     26.291      0.000       0.327       0.380
zip_98058         0.1956      0.013     14.895   

In [70]:
coef_df[coef_df['Column'] == "wat_YES"]

,Column,Value
82,wat_YES,0.526085


In [71]:



def calculate_price (sqft_living, sqft_lot, basement, yr_built, zipcode, grade, waterfront, \
                     view, mean_price, coef_df=coef_df, output='yes'): #output='yes'):
    if view == '-':
        view='NONE'

    if waterfront == 'YES': waterfront='WATERFRONT'
    else: waterfront = 'NONE'

    b0,b1,b2,b3,b4,b5,b6,b7,b8 = get_coeff( yr_built, zipcode, grade, waterfront, view, coef_df)
    y=round( np.exp(b0 + b1*np.log(sqft_living) + b2*np.log(sqft_lot)  + b3 + b4 + b5 + b6 + b7 +b7+b8) )

    y=y*(mean_price/mean_price_2014_2015)
    if output == 'yes': print('{:,.0f}'.format(y))
    return y,b0,b1,b2,b3,b4,b5,b6,b7,b8



abs_test_results=[]
test_results=[]
for i in range(100):
    X=np.array(df[['sqft_living','sqft_lot', 'basement', 'yr_built','zipcode','grade','waterfront','view', 'price']].sample())[0]
    sqft_living, sqft_lot, basement, yr_built, zipcode, grade, waterfront, view ,price=X

#    b0,b1,b2,b3,b4,b5,b6,b7 = get_coeff( year=year, zipcode=zipcode, grade=grade, water=water, view=view, coef_df=coef_df)
#    y=round( np.exp(b0 + b1*np.log(sqft_living) + b2 + b3 + b4 + b5 + b6 + b7*np.log(sqft_lot)) )

    y,b0,b1,b2,b3,b4,b5,b6,b7,b8= calculate_price (sqft_living, sqft_lot, basement, yr_built, zipcode, \
                                                   grade, waterfront, view, mean_price, coef_df=coef_df, output='no')

    miss_abs= np.abs(100-(y/price*100))
    miss= 100-(y/price*100)
    test_results.append(miss)
    abs_test_results.append(miss_abs)

avg_miss=sum(test_results)/len(test_results)
avg_miss_abs=sum(abs_test_results)/len(abs_test_results)
print (avg_miss, avg_miss_abs)


-0.27705302125594344 13.430633793367416


In [72]:
#print (X)
print(y,b0,b1,b2,b3,b4,b5,b6,b7,b8)
print (len(test_results))

636665.6488796031 9.009306472364264 0.4567834012190797 0.07578454862818891 -0.020399110360911354 -0.12505837566371925 0.5763376353723886 -0.34786102632076954 0 0.011270778581614516
100


In [73]:
title=form_items[0].children[0]
meanW=form_items[1].children[0]
zipW=form_items[2].children[1]
yearW=form_items[2].children[2]
gradeW=form_items[3].children[1]
livingW=form_items[4].children[1]
basementW=form_items[4].children[2]
lotW=form_items[5].children[1]
viewW=form_items[6].children[1]
waterW=form_items[6].children[2]

#buttonW=form_items[5].children[0]

output = widgets.interactive_output(calculate_price, { 'view':viewW,'waterfront':waterW,
    'zipcode': zipW, 'yr_built':yearW, 'grade':gradeW, 'basement':basementW, "mean_price":meanW,
    'sqft_living':livingW,'sqft_lot':lotW, } )



ui = widgets.VBox([form, output])

output.layout={'border': '3px solid green', 'width':'150px'}

#predictor=display(ui)


In [74]:
display(ui)